In [160]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, array, explode, sum, count
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from datetime import datetime

SYMBOLS_LIST = ['BTC.X', 'BSV.X', 'BCH.X', 'LTC.X', 'ETH.X', 'DOGE.X']

# NOTE: The environment needs to have scala installed for this to work
spark = SparkSession \
.builder \
.appName("myApp") \
.config("spark.mongodb.input.uri", "mongodb://127.0.0.1/cryptoracle") \
.config("spark.mongodb.input.collection", "messages") \
.config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.4.0') \
.getOrCreate()

messages_df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()

In [161]:
messages_df.count()

300

In [162]:
messages_df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- body: string (nullable = true)
 |-- conversation: struct (nullable = true)
 |    |-- parent_message_id: integer (nullable = true)
 |    |-- in_reply_to_message_id: null (nullable = true)
 |    |-- parent: boolean (nullable = true)
 |    |-- replies: integer (nullable = true)
 |-- created_at: string (nullable = true)
 |-- entities: struct (nullable = true)
 |    |-- chart: struct (nullable = true)
 |    |    |-- thumb: string (nullable = true)
 |    |    |-- large: string (nullable = true)
 |    |    |-- original: string (nullable = true)
 |    |    |-- url: string (nullable = true)
 |    |-- sentiment: struct (nullable = true)
 |    |    |-- basic: string (nullable = true)
 |-- filters: struct (nullable = true)
 |    |-- day_counts: integer (nullable = true)
 |    |-- official_api: boolean (nullable = true)
 |-- id: integer (nullable = true)
 |-- likes: struct (nullable = true)
 |    |-- total: integer

In [163]:
messages_df.first()

Row(_id=Row(oid='5e85e12a2dc1515fd0260a61'), body='$BTC.X BTC will hit 100k one day, no doubt about it.  The real question is how much purchasing power will the dollar lose by then', conversation=None, created_at='2020-03-25T00:31:19Z', entities=Row(chart=None, sentiment=Row(basic='Bullish')), filters=Row(day_counts=1, official_api=True), id=202506508, likes=None, links=None, mentioned_users=[], reshares=None, source=Row(id=2095, title='StockTwits For Android ', url='http://www.stocktwits.com/mobile'), symbols=[Row(aliases=['BTCUSD'], id=11418, is_following=False, symbol='BTC.X', title='Bitcoin BTC/USD', watchlist_count=157424)], user=Row(avatar_url='http://avatars.stocktwits.com/images/default_avatar_thumb.jpg', avatar_url_ssl='https://s3.amazonaws.com/st-avatars/images/default_avatar_thumb.jpg', classification=[], followers=1, following=0, id=3162884, ideas=86, identity='User', join_date='2020-03-08', like_count=42, name='gufyvyvyvyvyvtv', official=False, plus_tier='', premium_room='

### Create new dataframe with only required columns and filtered rows

In [164]:
messages_df = messages_df.select(messages_df['_id']['oid'].alias('_id'),
                   messages_df['body'],
                   messages_df['created_at'],
                   messages_df['entities']['sentiment']['basic'].alias('sentiment'),
                   messages_df['symbols']['symbol'].alias('symbols'),
                   messages_df['likes']['total'].alias('likes'),
                   messages_df['reshares']['reshared_count'].alias('reshares'))

def sum_interactions(likes, reshares):
    _sum = 1
    if likes:
        _sum += likes
    if reshares:
        _sum += reshares
    return _sum

def convert_sentiment(sentiment):
    if sentiment == "Bullish":
        return 2
    if sentiment == "Bearish":
        return 1
    return 0

def convert_date(dt):
    return datetime.strptime(dt, '%Y-%m-%dT%H:%M:%SZ').strftime('%Y-%m-%d-%H')

interactionUdf = udf(sum_interactions, IntegerType())
sentimentUdf = udf(convert_sentiment, IntegerType())
dateUdf = udf(convert_date, StringType())

messages_df = messages_df \
                .withColumn('interaction_count', interactionUdf('likes', 'reshares')) \
                .withColumn('sentiment', sentimentUdf('sentiment')) \
                .withColumn('created_at', dateUdf('created_at'))

In [165]:
messages_df.first()

Row(_id='5e85e12a2dc1515fd0260a61', body='$BTC.X BTC will hit 100k one day, no doubt about it.  The real question is how much purchasing power will the dollar lose by then', created_at='2020-03-25-00', sentiment=2, symbols=['BTC.X'], likes=None, reshares=None, interaction_count=1)

#### Distribution of sentiments

In [166]:
messages_df.groupBy('sentiment').count().collect()

[Row(sentiment=1, count=16),
 Row(sentiment=2, count=120),
 Row(sentiment=0, count=164)]

### Windowing


#### Unwrap array of symbols to new rows

In [167]:
# Since tweets can be attributed to more than one symbol, we unwrap the list into more rows
# this is okay as our final calculation will be grouped by symbol among other things

def weight_sentiment(sentiment, count):
    if sentiment == 0:
        return 0
    return sentiment * count

weightedSentimentUdf = udf(weight_sentiment, IntegerType())

messages_df = messages_df \
                .withColumn('weighted_sentiment', weightedSentimentUdf('sentiment', 'interaction_count')) \
                .withColumn('symbol', explode(messages_df['symbols']))
# filter to only those symbols that we care about
messages_df = messages_df.where(messages_df['symbol'].isin(SYMBOLS_LIST))

messages_df.groupby('symbol').count().collect()

[Row(symbol='BCH.X', count=46),
 Row(symbol='BTC.X', count=122),
 Row(symbol='ETH.X', count=68),
 Row(symbol='DOGE.X', count=30),
 Row(symbol='BSV.X', count=38),
 Row(symbol='LTC.X', count=45)]

In [168]:
messages_df.first()

Row(_id='5e85e12a2dc1515fd0260a61', body='$BTC.X BTC will hit 100k one day, no doubt about it.  The real question is how much purchasing power will the dollar lose by then', created_at='2020-03-25-00', sentiment=2, symbols=['BTC.X'], likes=None, reshares=None, interaction_count=1, weighted_sentiment=2, symbol='BTC.X')

In [169]:
grouped_df = messages_df.groupby(['created_at', 'symbol']).agg(
    sum('interaction_count').alias('sum_interaction_count'),
    sum('weighted_sentiment').alias('sum_weighted_sentiment'),
    count('_id').alias('volume_tweets'))

grouped_df = grouped_df.withColumn('overall_sentiment', grouped_df['sum_weighted_sentiment'] / grouped_df['sum_interaction_count'])


In [176]:
grouped_df.first()

Row(created_at='2020-03-25-00', symbol='ETH.X', sum_interaction_count=4, sum_weighted_sentiment=8, volume_tweets=2, overall_sentiment=2.0)

In [175]:
grouped_df.count()

170

In [91]:
messages_df.count()

349

In [182]:
joined_df = messages_df.join(grouped_df, ['created_at', 'symbol'])

In [ ]:
spark.stop()
